<a href="https://colab.research.google.com/github/Morningspread/Angry-Tweeter/blob/main/Fear_inference_RNN_Dec10%2C2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is our inference notebook 
## INference Dataset: Badhresh Savani - emotion dataset 

dataset location: [Tweet Emotion Dataset](https://github.com/dair-ai/emotion_dataset)

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#To do: 

1.   What parts are necessary / unecessary? List items
2.   How do we bring in other datasources? 
3.   Do we have to...vectorize the tweets...
4.   What about the labels? There's some mislabelling happening... 

#Resources 

https://www.tensorflow.org/guide/keras/save_and_serialize

model = ...  # Get model (Sequential, Functional Model, or Model subclass)
model.save('path/to/location')

## 1. Installing Hugging Face's nlp package

1.   This is needed for importing the Bhadresh Savani dataset... 
2.   There is another dependency in the import section... 



In [16]:
#Keep this until we switch out the Bhadresh Savani dataset... 

!pip install nlp
!pip install datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 2. Importing the libraries

In [17]:
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import nlp
import random
import pandas as pd

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
#Let's connect to Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 3. Importing the Dataset

1.   This section could be replaced with another import file.. 
2.   Canonical workflow: CSV import -> DataFrame -> List of Tweets -> Tokenization



In [20]:
dataset = nlp.load_dataset('emotion')

ConnectionError: ignored

In [ ]:
train = dataset['train']
val = dataset['validation']
test = dataset['test']

In [ ]:
#Object type: nlp.arrow_dataset.Dataset

type(train)

In [ ]:
#Let's see about converting this into a pandas dataframe

df = pd.DataFrame(data=train)

#Let's look at the shape: 

print(df.shape)

print(type(df))

df.head()

In [ ]:
#Extracting the df to a list... and then use this as an input into a tokenizer... 
#This seems to work: list with length of 16,000 

#tweetcol= df['text'].tolist()

In [ ]:
#Function: function argument "data" is an arrow dataset
#Output: the ouptut of the function is a list.. .

def get_tweets(data):
    tweets = [x['text'] for x in data]
    labels = [x['label'] for x in data]
    return tweets, labels

In [ ]:
#The function runs on an arrow objct but not a dataframe... 

tweets, labels = get_tweets(train)

#The function errors on the pandas dataframe object... 

#tweets, labels = get_tweets(df)


In [ ]:
#Question: What is the tweet and the labels object? 
#tweets -> train is a list (16,000) -> labels: a list (16,000)

labels?

In [ ]:
tweets[10], labels[10]

## 4. Tokenizing the Tweets

# Input into the tokenizer appears to be a list... verify... 

#To do - Nov 27, 2022 


1.   The input into the tokenizer appears to be a list... 
2.   What are the "acceptable" ranges for the num_words argument? I suppose that we could just play with it. 



#Importing the tokenizer and pad sequences... 

In [ ]:
#Importing the tokenizer... 

from tensorflow.keras.preprocessing.text import Tokenizer

#Importing the pad sequence... 

from tensorflow.keras.preprocessing.sequence import pad_sequences

#Instantiating the tokenizer object...

It's a: "keras.preprocessing.text.Tokenizer"

In [ ]:
#Hyperparameter: the num_words variable...not sure what 

tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>')

#tokenizing works with either: "tweets" -> arrowdataframe or the "tweetcol" -> Pandas dataframe?

tokenizer.fit_on_texts(tweets)

#Testing out the tokenizer... 

print(tokenizer.texts_to_sequences([tweets[10]]))

In [ ]:
#What is the tweet object? 
#It's a list and each tweet is split by commas and '' 

#tweets?

#Visualizing the lengths of the tweets - this can vary depending on text extraction...

In [ ]:
#For loop for creating the lenghts of the tweets 
#I'm not really sure how this works except that 

lengths = [len(t.split(' ')) for t in tweets]

#Visualizing the tweet length - create a set object

plt.hist(lengths, bins=len(set(lengths)))
plt.show()

In [ ]:
#What is lengths? It's a list... 
#Let's try to convert to a dataframe...

a=pd.DataFrame(lengths)
a.describe()

## 5. Tokeninzing, Truncating and Padding Sequences



In [ ]:
#The function that tokenizes and pads the tweets... 
#The function seems to eb taking two arguments: "tokenizer" and "tweets"

def get_sequences(tokenizer, tweets):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=50, padding='post')
    return padded_sequences

In [ ]:
#Runing the "get_sequences" tokenizer...

padded_train_sequences = get_sequences(tokenizer, tweets)

In [ ]:
padded_train_sequences[10]

## 6. Preparing the Labels

Nov 28, 2022: This is the part that doesn't make sense... 


In [ ]:
#Where is this object getting pulled from? It's from the function... 
#And that function is extracting from the dataset that that we no longer need... 

classes = set(labels)
print(classes)

In [ ]:
#So, then what is the classes object? 

print(type(classes))

In [ ]:
#Looking at the distribution  of the data

plt.hist(labels, bins=11)
plt.show()

In [ ]:
classes_to_index = dict((c, i) for i, c in enumerate(classes))
index_to_classes = dict((v, k) for k, v in classes_to_index.items())

In [ ]:
classes_to_index

In [ ]:
index_to_classes

In [ ]:
names_to_ids = lambda labels: np.array([classes_to_index.get(x) for x in labels])

In [ ]:
train_labels = names_to_ids(labels)
print(train_labels[10])

In [ ]:
#So what is train_labels then? 

train_labels?

## 8. Inference

In [ ]:
#loading the trained model - Robin's version 
#from tensorflow import keras
#model = keras.models.load_model("/content/drive/MyDrive/my_model")

In [ ]:
#Loading the model -> Anger Arousal - Oct 20, 2022 
#I'm not sure about this -> are we loading the right model...


model = keras.models.load_model("/content/drive/MyDrive/Sean/Emoclass_Dec2022/Fear_arousal/Fear_Arousal_Model_6cat_dec10,2022")

In [ ]:
#I'm not sure where this thing came from...I think it was Robin...
#But I don't think that we want to use test tweets 

#test_tweets, test_labels = get_tweets(test)
#test_sequences = get_sequences(tokenizer, test_tweets)
#test_labels = names_to_ids(test_labels)


#So let's duplicate the code and run it down here... 

train_tweets, test_labels = get_tweets(train)
train_sequences = get_sequences(tokenizer, train_tweets)
train_labels = names_to_ids(train_labels)

Inference

In [ ]:
#preds = model.predict_classes(test_sequences)
preds=model.predict(train_sequences) 
classes_x=np.argmax(preds,axis=1)
preds.shape, train_labels.shape

In [ ]:
inf_df=pd.DataFrame({
    'data':train_tweets,
    "labels_predicted": classes_x                    
})
inf_df["label_marked"]=inf_df['labels_predicted'].apply(lambda x: index_to_classes[x])
inf_df

In [ ]:
#What does our inf_df object look like? 

inf_df.shape

#We have an inference dataset... 

Oct 20, 2022: Before we dive into anything here...let's see what we have wtr to classifications / classes. This is one part of the code that doesn't make a lot of sense to me -> i.e. I need to better understand it. 

In [ ]:
#Let's see what it looks like...

inf_df.head()

In [ ]:
inf_df['labels_predicted'].value_counts()

In [ ]:
#Let's look at the frequency of these...REmember -> I don't think that this label is working very well...
inf_df['label_marked'].value_counts()

In [ ]:
#Let's export the dataset... 

/content/drive/MyDrive/Sean/Emoclass_Dec2022/Fear_arousal

filepath = =('/content/drive/MyDrive/Sean/Emoclass_Dec2022/Fear_arousal/Fear_arousal_inference_bsavani_Dec10,2022') 

df.to_csv(filepath)  


#Note Dec 5, 2022 
## We may add some diagnostic code to this workbook. But for now, let's just work with this codebook. As a general strategy, we should be focusing on making codebooks more concise / simpler... 